## Oracle Cloud Instance POC


In [1]:
import os
import json
from pathlib import Path
from typing import Dict
from dataclasses import dataclass


import bravado
from bravado.client import SwaggerClient, CallableOperation
# from bravado.requests_client import RequestsClient
from gc3_query.lib.bravado.requests_client import OCRequestsClient
from bravado.requests_client import RequestsResponseAdapter
from bravado.swagger_model import load_file
from bravado_core.exception import MatchingResponseNotFound
from bravado.exception import HTTPBadRequest
from bravado.http_future import HttpFuture

# from tinydb import TinyDB, Query
import mongoengine

# from prettyprinter import pprint, pformat
from pprint import pprint, pformat

## https://medium.com/@betz.mark/validate-json-models-with-swagger-and-bravado-5fad6b21a825
# Validate json models with swagger and bravado
from bravado_core.spec import Spec

from sortedcontainers.sorteddict import SortedDict


In [2]:
from gc3_query.lib import List, Optional, Any, Callable, Dict, Tuple, Union, Set, Generator, Path

In [3]:
# from secrets import opc_username, opc_password
import keyring

opc_username = "eric.harris@oracle.com"
opc_password = keyring.get_password("OPC", "gc30003")


In [4]:
idm_domain_name = 'gc30003'
idm_service_instance_id = '587626604'
iaas_rest_endpoint = r'https://compute.uscom-central-1.oraclecloud.com'
iaas_auth_endpoint = f'{iaas_rest_endpoint}/authenticate/'

print(f'iaas_rest_endpoint: {iaas_rest_endpoint}')
print(f'iaas_auth_endpoint: {iaas_auth_endpoint}\n')


### Username/pass setup
idm_domain_username = f'/Compute-{idm_domain_name}/{opc_username}'
idm_service_instance_username = f'/Compute-{idm_service_instance_id}/{opc_username}'
# username = traditional_iaas_username
username = idm_service_instance_username
# basic_auth_cred = _basic_auth_str(username, opc_password)


iaas_rest_endpoint: https://compute.uscom-central-1.oraclecloud.com
iaas_auth_endpoint: https://compute.uscom-central-1.oraclecloud.com/authenticate/



In [5]:
json_data = {"user": username, "password": opc_password}
print(f'idm_domain_username: {idm_domain_username}')
print(f'idm_service_instance_username: {idm_service_instance_username}')
print(f'username: {username}')

idm_domain_username: /Compute-gc30003/eric.harris@oracle.com
idm_service_instance_username: /Compute-587626604/eric.harris@oracle.com
username: /Compute-587626604/eric.harris@oracle.com


In [6]:
proxies = {
  'http': 'http://www-proxy-ash7.us.oracle.com:80',
  'https': 'https://www-proxy-ash7.us.oracle.com:80',
}

#### Create DB to store Instance data

In [7]:
# instances_db = TinyDB('instances.tdb.json')

####  Authenticate and get header token


In [8]:
headers = dict([('Content-Type', 'application/oracle-compute-v3+json'),
                ('Accept', 'application/oracle-compute-v3+directory+json'),
                ])
requests_client = OCRequestsClient()
requests_client.session.headers.update(headers)
requests_client.session.proxies.update(proxies)


# print(f"requests_client.session.headers before update: {requests_client.session.headers}\n")
requests_client.session.headers.update(headers)
# print(f"requests_client.session.headers after update: {requests_client.session.headers}\n")

response = requests_client.session.post(url=iaas_auth_endpoint, json=json_data)
print(f'Response OK: {response.ok}, Status Code: {response.status_code}, URL: {response.url}')
if response.ok and 'Set-Cookie' in response.headers:
    print(f"Auth request succeess.\n")
    ### The auth cookie is already placed in the session ... nothing else needs to be done.
#     print(f"\nSession Cookies: {requests_client.session.cookies}")
#     print(f"\nResponse Headers['Set-Cookie']: {response.headers['Set-Cookie']}")
else:
    print(f'Something failed! Response OK: {response.ok}, Status Code: {response.status_code}')

# print(f"requests_client.session.headers before update: {requests_client.session.headers}\n")
cookie_header = {'Cookie': response.headers['Set-Cookie']}
# print(f"cookie_header: {cookie_header}\n")
requests_client.session.headers.update(cookie_header)
# print(f"requests_client.session.headers after update: {requests_client.session.headers}\n")


Response OK: True, Status Code: 204, URL: https://compute.uscom-central-1.oraclecloud.com/authenticate/
Auth request succeess.



In [9]:
## Update the swagger spec to use https
spec_file_path = Path().joinpath('open_api_definitions/iaas_instances.json').resolve()
spec_dict = load_file(spec_file_path)
spec_dict['schemes'] = ['https']


In [10]:
headers = dict([('Content-Type', 'application/oracle-compute-v3+json'),
                ('Accept', 'application/oracle-compute-v3+json, application/oracle-compute-v3+directory+json, json, text/html')
                ])

# headers = dict([('Content-Type', 'application/oracle-compute-v3+json'),
#                 ('Accept', 'application/oracle-compute-v3+json, json, text/html')])

In [11]:
requests_client = OCRequestsClient()
requests_client.session.headers.update(headers)
requests_client.session.proxies.update(proxies)
requests_client.session.headers.update(cookie_header)
swagger_client = SwaggerClient.from_spec(spec_dict=spec_dict,
                                         origin_url=iaas_rest_endpoint,
                                         http_client=requests_client,
                                         config={'also_return_response': True,
                                                 'validate_responses': True,
                                                 'validate_requests': True,
                                                 'validate_swagger_spec': True})
                                         # config={'also_return_response': True,
                                         #         'validate_responses': False,
                                         #         'validate_requests': True,
                                         #         'validate_swagger_spec': True})
#                                          config={'also_return_response': True,
#                                                  'validate_responses': False,
#                                                  'validate_requests': False,
#                                                  'validate_swagger_spec': False})
instances_resource = swagger_client.Instances

In [12]:
for operation_name in dir(instances_resource):
    callable_operation = getattr(instances_resource, operation_name)
    print(f"{callable_operation.operation.operation_id}\t\t http_method={callable_operation.operation.http_method}\tpath={callable_operation.operation.path_name}")
    
    

deleteInstance		 http_method=delete	path=/instance/{name}
discoverInstance		 http_method=get	path=/instance/{container}
discoverRootInstance		 http_method=get	path=/instance/
getInstance		 http_method=get	path=/instance/{name}
listInstance		 http_method=get	path=/instance/{container}/
updateInstance		 http_method=put	path=/instance/{name}


In [13]:
callable_operation

### discoverRootInstance

<pre>
{
  "swagger" : "2.0",
  "info" : {
    "version" : "17.4.2-20171207.013930",
    "description" : "A Compute Classic instance is a virtual machine running a specific operating system and with CPU and memory resources that you specify. See <a target=\"_blank\" href=\"http://www.oracle.com/pls/topic/lookup?ctx=stcomputecs&id=STCSG-GUID-F928F362-2DB6-4E45-843F-C269E0740A36\">About Instances</a> in <em>Using Oracle Cloud Infrastructure Compute Classic</em>.<p>You can view and delete instances using the HTTP requests listed below.",
    "title" : "Instances"
  },
  "schemes" : [ "http" ],
  "consumes" : [ "application/oracle-compute-v3+json", "application/oracle-compute-v3+directory+json" ],
  "produces" : [ "application/oracle-compute-v3+json", "application/oracle-compute-v3+directory+json" ],
 <b><font color="red"> "paths" : { </font></b>
 <b>    "/instance/" : {</b>
 <b><font color="red">     "get" </font></b> : {
        "tags" : [ "Instances" ],
        "summary" : "Retrieve Names of Containers",
        "description" : "Retrieves the names of containers that contain objects that you can access. You can use this information to construct the multipart name of an object.<p>Required Role:To complete this task, you must have the <code>Compute_Monitor</code> or <code>Compute_Operations</code> role. If this role isn't assigned to you or you're not sure, then ask your system administrator to ensure that the role is assigned to you in Oracle Cloud My Services. See <a target=\"_blank\" href=\"http://www.oracle.com/pls/topic/lookup?ctx=stcomputecs&id=MMOCS-GUID-54C2E747-7D5B-451C-A39C-77936178EBB6\">Modifying User Roles</a> in <em>Managing and Monitoring Oracle Cloud</em>.",
      "operationId" : "<b><font color="red">discoverRootInstance </font></b>",
    <b><font color="red">   "responses"</font> : { </b>
     <b>     "200" : { </b>
            "headers" : {
     <b>         "set-cookie" : { </b>
     <b>           "type" : "string", </b>
      <b>          "description" : "The cookie value is returned if the session is extended" </b>
              }
            },
            "description" : "OK. See <a class=\"xref\" href=\"Status%20Codes.html\">Status Codes</a> for information about other possible HTTP status codes.",
            "schema" : {
              <b>"$ref" : "#/definitions/Instance-discover-response"</b>
            }
          }
        },
        "consumes" : [ "application/oracle-compute-v3+json" ],
        "produces" : [ "application/oracle-compute-v3+directory+json" ],
     <font color="red">   "parameters" </font>: [ {
         <b> "name" : "Cookie",</b>
          "in" : "header",
          "type" : "string",
          "description" : "The Cookie: header must be included with every request to the service. It must be set to the value of the set-cookie header in the response received to the POST /authenticate/ call."
        } ]
      }
    },
.
.
.
<font color="red">  "definitions" </font>: {
.
.
.

    "Instance-discover-response" : {
      "properties" : {
        "result" : {
          "items" : {
            "type" : "string"
          },
          "type" : "array"
        }
      }
    },
</pre>










In [14]:

@dataclass
class GetOpReturn:
    operation_details: Dict[str, Any]
    operation_response: RequestsResponseAdapter


def get_op(operation_id: str, **wkargs) -> Tuple[Dict[str, Any], RequestsResponseAdapter]:
    print(f"Operation {operation_id} starting.")
    try:
        operation_future = getattr(instances_resource, operation_id)(**wkargs)
        url = operation_future.future.request.url
        print(f"REST url for {operation_id}: {url}")
        operation_result, operation_response = operation_future.result()
    except HTTPBadRequest:
        print("Request failed for {operation_id}! ")
        print(f"URL: {operation_future.future.request.url}")
        raise
    operation_details = json.loads(operation_result)
    return GetOpReturn(operation_details,operation_response )
#     print("\n{} operation_details:\nHTTP method: {}\nAPI url: {}:\n {}\n".format(operation_id, operation_future.operation.http_method, url, pformat(operation_details)))
#     print(f"Operation {operation_id} finished.\n")


In [15]:
operation_id = "discoverRootInstance"
print(f"Operation {operation_id} starting.")
try:
    operation_future = getattr(instances_resource, operation_id)()
    url = operation_future.future.request.url
    print(f"REST url for {operation_id}: {url}")
    operation_result, operation_response = operation_future.result()
except HTTPBadRequest:
    print("Request failed for {operation_id}! ")
    print(f"URL: {operation_future.future.request.url}")
    raise
operation_details = json.loads(operation_result)
print("\n{} operation_details:\nHTTP method: {}\nAPI url: {}:\n {}\n".format(operation_id, operation_future.operation.http_method, url, pformat(operation_details)))
print(f"Operation {operation_id} finished.\n")

Operation discoverRootInstance starting.
[2018-05-18 15:36:37.761236] DEBUG: gc3_query.lib.bravado.requests_client: input request_params={'method': 'GET', 'url': 'https://compute.uscom-central-1.oraclecloud.com/instance/', 'params': {}, 'headers': {}}
[2018-05-18 15:36:37.762418] DEBUG: gc3_query.lib.bravado.requests_client: url=https://compute.uscom-central-1.oraclecloud.com/instance/ parsed_url=ParseResult(scheme='https', netloc='compute.uscom-central-1.oraclecloud.com', path='/instance/', params='', query='', fragment=''), unqoted_path=/instance/, requoted_path=/instance/, new_url=https://compute.uscom-central-1.oraclecloud.com/instance/
[2018-05-18 15:36:37.765230] DEBUG: gc3_query.lib.bravado.requests_client: returned request_params={'method': 'GET', 'url': 'https://compute.uscom-central-1.oraclecloud.com/instance/', 'params': {}, 'headers': {}}
REST url for discoverRootInstance: https://compute.uscom-central-1.oraclecloud.com/instance/

discoverRootInstance operation_details:
HTT

In [16]:
operation_id = "discoverInstance"
print(f"Operation {operation_id} starting.")
try:
    # API url: https://compute.uscom-central-1.oraclecloud.com/instance/Compute-587626604
    container = operation_details['result'][0].lstrip('/').rstrip('/')
    # # API url: https://compute.uscom-central-1.oraclecloud.com/instance/Compute-587626604/eric.harris@oracle.com
    # container = f"{container}/{opc_username}"
    # # API url: https://compute.uscom-central-1.oraclecloud.com/instance/Compute-587626604/eric.harris@oracle.com/
    container = f"{container}/{opc_username}/"
    print(f"container: {container}")
    operation_future = getattr(instances_resource, operation_id)(container=container)
    url = operation_future.future.request.url
    print(f"REST url for {operation_id}: {url}")
    operation_result, operation_response = operation_future.result()
except HTTPBadRequest:
    print("Request failed for {operation_id}! ")
    print(f"URL: {operation_future.future.request.url}")
    raise
operation_details = json.loads(operation_result)
print("\n{} operation_details:\nHTTP method: {}\nAPI url: {}:\n {}\n".format(operation_id, operation_future.operation.http_method, url, pformat(operation_details)))
print(f"Operation {operation_id} finished.\n")

Operation discoverInstance starting.
container: Compute-587626604/eric.harris@oracle.com/
[2018-05-18 15:36:58.168999] DEBUG: gc3_query.lib.bravado.requests_client: input request_params={'method': 'GET', 'url': 'https://compute.uscom-central-1.oraclecloud.com/instance/Compute-587626604%2Feric.harris%40oracle.com%2F', 'params': {}, 'headers': {}}
[2018-05-18 15:36:58.171245] DEBUG: gc3_query.lib.bravado.requests_client: url=https://compute.uscom-central-1.oraclecloud.com/instance/Compute-587626604%2Feric.harris%40oracle.com%2F parsed_url=ParseResult(scheme='https', netloc='compute.uscom-central-1.oraclecloud.com', path='/instance/Compute-587626604%2Feric.harris%40oracle.com%2F', params='', query='', fragment=''), unqoted_path=/instance/Compute-587626604/eric.harris@oracle.com/, requoted_path=/instance/Compute-587626604/eric.harris%40oracle.com/, new_url=https://compute.uscom-central-1.oraclecloud.com/instance/Compute-587626604/eric.harris@oracle.com/
[2018-05-18 15:36:58.171937] DEBUG: 

In [17]:
operation_id = "listInstance"
# container = 'Compute-587626604/eric.harris@oracle.com/GC3NAAC-CDMT-LWS1'
container = 'Compute-587626604/eric.harris@oracle.com/dbaas/gc3-naac-cdmt-d05-jcsdb/db_1/vm-1'
print(f"Operation {operation_id} starting.")
try:
    # container = operation_details['result'][0].lstrip('/').rstrip('/')
    # container = 'Compute-587626604/eric.harris@oracle.com'
    # container = 'Compute-587626604/eric.harris@oracle.com/GC3NAAC-CDMT-LWS1'
    print(f"container: {container}")
    operation_future = getattr(instances_resource, operation_id)(container=container)
    url = operation_future.future.request.url
    print(f"REST url for {operation_id}: {url}")
    operation_result, operation_response = operation_future.result()
except HTTPBadRequest:
    print("Request failed for {operation_id}! ")
    print(f"URL: {operation_future.future.request.url}")
    raise
operation_details = json.loads(operation_result)
print("\n{} operation_details:\nHTTP method: {}\nAPI url: {}:\n {}\n".format(operation_id, operation_future.operation.http_method, url, pformat(operation_details)))
print(f"Operation {operation_id} finished.")

Operation listInstance starting.
container: Compute-587626604/eric.harris@oracle.com/dbaas/gc3-naac-cdmt-d05-jcsdb/db_1/vm-1
[2018-05-18 15:36:59.282393] DEBUG: gc3_query.lib.bravado.requests_client: input request_params={'method': 'GET', 'url': 'https://compute.uscom-central-1.oraclecloud.com/instance/Compute-587626604%2Feric.harris%40oracle.com%2Fdbaas%2Fgc3-naac-cdmt-d05-jcsdb%2Fdb_1%2Fvm-1/', 'params': {}, 'headers': {}}
[2018-05-18 15:36:59.283911] DEBUG: gc3_query.lib.bravado.requests_client: url=https://compute.uscom-central-1.oraclecloud.com/instance/Compute-587626604%2Feric.harris%40oracle.com%2Fdbaas%2Fgc3-naac-cdmt-d05-jcsdb%2Fdb_1%2Fvm-1/ parsed_url=ParseResult(scheme='https', netloc='compute.uscom-central-1.oraclecloud.com', path='/instance/Compute-587626604%2Feric.harris%40oracle.com%2Fdbaas%2Fgc3-naac-cdmt-d05-jcsdb%2Fdb_1%2Fvm-1/', params='', query='', fragment=''), unqoted_path=/instance/Compute-587626604/eric.harris@oracle.com/dbaas/gc3-naac-cdmt-d05-jcsdb/db_1/vm-1/,

In [18]:
operation_details

{'result': ['/Compute-587626604/eric.harris@oracle.com/dbaas/gc3-naac-cdmt-d05-jcsdb/db_1/vm-1/963e740b-052b-469c-849c-37a1d0752b7a']}

#### Back to our example

<pre>
(psmcli) [root@eharris-lnxobi-01 ~]#<b> opc -f json compute instance list -h </b>
NAME:
   compute instance list - <font color="red">Retrieve Details of all Instances in a Container</font>

USAGE:
   compute instance list container [options...]

DESCRIPTION:
   Retrieve Details of all Instances in a Container

REQUIRED ARGUMENTS:
   container - /Compute-identity_domain/user or    /Compute-identity_domain

OPTIONS:
   --availability-domain value  The availability domain the instance is in
   --tags value                 Strings used to tag the instance. When you specify tags, only instances tagged with the specified value are displayed.

(psmcli) [root@eharris-lnxobi-01 ~]#


<h4>Instances Swagger Definition</h4>


{
  "swagger" : "2.0",
  "info" : {
    "version" : "17.4.2-20171207.013930",
    "description" : "A Compute Classic instance is a virtual machine running a specific operating system and with CPU and memory resources that you specify. See <a target=\"_blank\" href=\"http://www.oracle.com/pls/topic/lookup?ctx=stcomputecs&id=STCSG-GUID-F928F362-2DB6-4E45-843F-C269E0740A36\">About Instances</a> in <em>Using Oracle Cloud Infrastructure Compute Classic</em>.<p>You can view and delete instances using the HTTP requests listed below.",
    "title" : "Instances"
  },
  "schemes" : [ "https" ],
  "consumes" : [ "application/oracle-compute-v3+json", "application/oracle-compute-v3+directory+json" ],
  "produces" : [ "application/oracle-compute-v3+json", "application/oracle-compute-v3+directory+json" ],
  "paths" : {
    .
    .
    .

    "/instance/{container}/" : {
      "get" : {
        "tags" : [ "Instances" ],
        "summary" : <font color="red">"Retrieve Details of all Instances in a Container",</font>
        "description" : "Retrieves details of the instances that are in the specified container and match the specified query criteria. If you don't specify any query criteria, then details of all the instances in the container are displayed. To filter the search results, you can pass one or more of the following query parameters, by appending them to the URI in the following syntax:<p><code>?parameter1=value1&ampparameter2=value2&ampparameterN=valueN</code><p><b>Required Role: </b>To complete this task, you must have the <code>Compute_Monitor</code> or <code>Compute_Operations</code> role. If this role isn't assigned to you or you're not sure, then ask your system administrator to ensure that the role is assigned to you in Oracle Cloud My Services. See <a target=\"_blank\" href=\"http://www.oracle.com/pls/topic/lookup?ctx=stcomputecs&id=MMOCS-GUID-54C2E747-7D5B-451C-A39C-77936178EBB6\">Modifying User Roles</a> in <em>Managing and Monitoring Oracle Cloud</em>.",
        "operationId" : "listInstance",
        "responses" : {
          "200" : {
            "headers" : {
              "set-cookie" : {
                "type" : "string",
                "description" : "The cookie value is returned if the session is extended"
              }
            },
            "description" : "OK. See <a class=\"xref\" href=\"Status%20Codes.html\">Status Codes</a> for information about other possible HTTP status codes.",
            "schema" : {
              "$ref" : "#/definitions/Instance-list-response"
            }
          }
        },
        "consumes" : [ "application/oracle-compute-v3+json" ],
        "produces" : [ "application/oracle-compute-v3+json" ],
        "parameters" : [ {
          "name" : "container",
          "in" : "path",
          "description" : "<code>/Compute-<em>identity_domain</em>/<em>user</em></code> or <p><code>/Compute-<em>identity_domain</em></code>",
          "required" : true,
          "type" : "string"
        }, {
          "name" : "availability_domain",
          "in" : "query",
          "description" : "The availability domain the instance is in",
          "required" : false,
          "type" : "string"
        }, {
          "name" : "tags",
          "in" : "query",
          "description" : "Strings used to tag the instance. When you specify tags, only instances tagged with the specified value are displayed.",
          "required" : false,
          "type" : "array",
          "items" : {
            "type" : "string"
          }
        }, {
          "name" : "Cookie",
          "in" : "header",
          "type" : "string",
          "description" : "The Cookie: header must be included with every request to the service. It must be set to the value of the set-cookie header in the response received to the POST /authenticate/ call."
        } ]
      }
    },




</pre>

In [19]:
operation_details['result']

['/Compute-587626604/eric.harris@oracle.com/dbaas/gc3-naac-cdmt-d05-jcsdb/db_1/vm-1/963e740b-052b-469c-849c-37a1d0752b7a']

In [20]:
headers = dict([('Content-Type', 'application/oracle-compute-v3+json'),
                ('Accept', 'application/oracle-compute-v3+json, json, text/html')])


requests_client = OCRequestsClient()
requests_client.session.headers.update(headers)
requests_client.session.proxies.update(proxies)
requests_client.session.headers.update(cookie_header)
swagger_client = SwaggerClient.from_spec(spec_dict=spec_dict,
                                         origin_url=iaas_rest_endpoint,
                                         http_client=requests_client,
                                         # config={'also_return_response': True,
                                         #         'validate_responses': True,
                                         #         'validate_requests': True,
                                         #         'validate_swagger_spec': True})
                                         # config={'also_return_response': True,
                                         #         'validate_responses': False,
                                         #         'validate_requests': True,
                                         #         'validate_swagger_spec': True})
                                         config={'also_return_response': True,
                                                 'validate_responses': False,
                                                 'validate_requests': False,
                                                 'validate_swagger_spec': False})
instances_resource = swagger_client.Instances



<pre>
    "Instance-response" : {
      "properties" : {
        "account" : {
          "type" : "string",
          "description" : "Shows the default account for your identity domain."
        },
        "attributes" : {
          "additionalProperties" : {
            "type" : "object"
          },
          "type" : "object",
          "description" : "A dictionary of attributes to be made available to the instance. A value with the key \"userdata\" will be made available in an EC2-compatible manner."
        },
        "availability_domain" : {
          "type" : "string",
          "description" : "The availability domain the instance is in"
        },
        "boot_order" : {
          "items" : {
            "type" : "integer"
          },
          "type" : "array",
          "description" : "Boot order list."
        },
        "desired_state" : {
          "type" : "string",
          "description" : "Desired state for the instance. The value can be <code>shutdown</code> or <code>running</code> to shutdown an instance or to restart a previously shutdown instance respectively."
        },
        "disk_attach" : {
          "type" : "string",
          "description" : "A label assigned by the user to identify disks."
        },
        "domain" : {
          "type" : "string",
          "description" : "The default domain to use for the hostname and for DNS lookups."
        },
        "entry" : {
          "type" : "integer",
          "description" : "Optional imagelistentry number (default will be used if not specified)."
        },
        "error_reason" : {
          "type" : "string",
          "description" : "The reason for the instance going to error state, if available."
        },
        "fingerprint" : {
          "type" : "string",
          "description" : "SSH server fingerprint presented by the instance."
        },
        "hostname" : {
          "type" : "string",
          "description" : "The hostname for this instance."
        },
        "hypervisor" : {
          "additionalProperties" : {
            "type" : "object"
          },
          "type" : "object",
          "description" : "A dictionary of hypervisor-specific attributes."
        },
        "image_format" : {
          "type" : "string",
          "description" : "The format of the image."
        },
        "imagelist" : {
          "type" : "string",
          "description" : "Name of imagelist to be launched."
        },
        "ip" : {
          "type" : "string",
          "description" : "IP address of the instance."
        },
        "label" : {
          "type" : "string",
          "description" : "A label assigned by the user, specifically for defining inter-instance relationships."
        },
        "name" : {
          "type" : "string",
          "description" : "Multipart name of the instance."
        },
        "networking" : {
          "additionalProperties" : {
            "type" : "object"
          },
          "type" : "object",
          "description" : "Mapping of <device name> to network specifiers for virtual NICs to be attached to this instance."
        },
        "placement_requirements" : {
          "items" : {
            "type" : "string"
          },
          "type" : "array",
          "description" : "A list of strings specifying arbitrary tags on nodes to be matched on placement."
        },
        "platform" : {
          "type" : "string",
          "description" : "The OS platform for the instance."
        },
        "priority" : {
          "type" : "string",
          "description" : "The priority at which this instance will be run."
        },
        "quota" : {
          "type" : "string",
          "description" : "Not used"
        },
        "relationships" : {
          "items" : {
            "additionalProperties" : {
              "type" : "object"
            },
            "type" : "object"
          },
          "type" : "array",
          "description" : "A list of relationship specifications to be satisfied on this instance's placement"
        },
        "resolvers" : {
          "items" : {
            "type" : "string"
          },
          "type" : "array",
          "description" : "Resolvers to use instead of the default resolvers."
        },
        "reverse_dns" : {
          "type" : "boolean",
          "description" : "Add PTR records for the hostname."
        },
        "shape" : {
          "type" : "string",
          "description" : "A shape is a resource profile that specifies the number of CPU threads and the amount of memory (in MB) to be allocated to an instance."
        },
        "sshkeys" : {
          "items" : {
            "type" : "string"
          },
          "type" : "array",
          "description" : "SSH keys that will be exposed to the instance."
        },
        "start_time" : {
          "type" : "string",
          "description" : "Start time of the instance."
        },
        "state" : {
          "type" : "string",
          "description" : "State of the instance."
        },
        "storage_attachments" : {
          "items" : {
            "type" : "string"
          },
          "type" : "array",
          "description" : "List of dictionaries containing storage attachment Information."
        },
        "tags" : {
          "items" : {
            "type" : "string"
          },
          "type" : "array",
          "description" : "Comma-separated list of strings used to tag the instance."
        },
        "uri" : {
          "type" : "string",
          "description" : "Uniform Resource Identifier"
        },
        "vcable_id" : {
          "type" : "string",
          "description" : "vCable for this instance."
        },
        "vnc" : {
          "type" : "string",
          "description" : "IP address and port of the VNC console for the instance."
        }
      }
    },

</pre>



In [21]:
operation_id = "getInstance"
print(f"\nOperation {operation_id} starting.")
try:
    name = operation_details['result'][0].lstrip('/').rstrip('/')
    print(f"name: {name}")
    operation_future: HttpFuture = getattr(instances_resource, operation_id)(name=name)
    url = operation_future.future.request.url
    print(f"REST url for {operation_id}: {url}")
    print(f"Accept Header=[{operation_future.future.session.headers['Accept']}], Content-Type Header=[{operation_future.future.session.headers['Content-Type']}]")
    operation_result, operation_response = operation_future.result()
except bravado.exception.HTTPBadRequest:
    print("Request failed for {operation_id}! ")
    print(f"URL: {operation_future.future.request.url}")
    raise
except bravado.exception.HTTPNotFound:
    print("Request failed for {operation_id}! ")
    print(f"URL: {operation_future.future.request.url}")
    raise
operation_details = json.loads(operation_result)
print("\n{} operation_details:\nHTTP method: {}\nAPI url: {}:\n {}\n".format(operation_id, operation_future.operation.http_method, url, pformat(operation_details)))
print(f"Operation {operation_id} finished.")


Operation getInstance starting.
name: Compute-587626604/eric.harris@oracle.com/dbaas/gc3-naac-cdmt-d05-jcsdb/db_1/vm-1/963e740b-052b-469c-849c-37a1d0752b7a
[2018-05-18 15:37:00.363025] DEBUG: gc3_query.lib.bravado.requests_client: input request_params={'method': 'GET', 'url': 'https://compute.uscom-central-1.oraclecloud.com/instance/Compute-587626604%2Feric.harris%40oracle.com%2Fdbaas%2Fgc3-naac-cdmt-d05-jcsdb%2Fdb_1%2Fvm-1%2F963e740b-052b-469c-849c-37a1d0752b7a', 'params': {}, 'headers': {}}
[2018-05-18 15:37:00.365078] DEBUG: gc3_query.lib.bravado.requests_client: url=https://compute.uscom-central-1.oraclecloud.com/instance/Compute-587626604%2Feric.harris%40oracle.com%2Fdbaas%2Fgc3-naac-cdmt-d05-jcsdb%2Fdb_1%2Fvm-1%2F963e740b-052b-469c-849c-37a1d0752b7a parsed_url=ParseResult(scheme='https', netloc='compute.uscom-central-1.oraclecloud.com', path='/instance/Compute-587626604%2Feric.harris%40oracle.com%2Fdbaas%2Fgc3-naac-cdmt-d05-jcsdb%2Fdb_1%2Fvm-1%2F963e740b-052b-469c-849c-37a1d0752

In [22]:
operation_details

{'domain': 'compute-587626604.oraclecloud.internal.',
 'placement_requirements': ['/system/compute/placement/default',
  '/oracle/compute/dedicated/fcbaed9f-242b-42ce-ac77-dd727f9710eb',
  '/system/compute/allow_instances'],
 'ip': '10.19.9.86',
 'fingerprint': '59:9b:36:df:e8:b4:02:e6:5c:5b:9f:7b:d4:33:65:ad',
 'image_metadata_bag': '/oracle/machineimage_metadata/0fa1013075c2462aa148af5958353099',
 'site': '',
 'shape': 'oc4',
 'imagelist': None,
 'image_format': 'raw',
 'relationships': [],
 'availability_domain': '/uscom-central-1a',
 'networking': {'eth0': {'model': '',
   'seclists': ['/Compute-587626604/eric.harris@oracle.com/dbaas/gc3-naac-cdmt-d05-jcsdb/db_1/ora_db',
    '/Compute-587626604/eric.harris@oracle.com/GC3NAACCDMT_PSFT'],
   'dns': ['gc3-naac-cdmt-d05-jcsdb.compute-587626604.oraclecloud.internal.'],
   'vethernet': '/oracle/public/default',
   'nat': 'ipreservation:/Compute-587626604/eric.harris@oracle.com/dbaas/gc3-naac-cdmt-d05-jcsdb/db_1/vm-1/ipreservation'}},
 's

In [23]:
operation_id = "getInstance"
print(f"\nOperation {operation_id} starting.")
try:
    name = operation_details['name'].lstrip('/').rstrip('/')
    print(f"name: {name}")
    operation_future: HttpFuture = getattr(instances_resource, operation_id)(name=name)
    url = operation_future.future.request.url
    print(f"REST url for {operation_id}: {url}")
    print(f"Accept Header=[{operation_future.future.session.headers['Accept']}], Content-Type Header=[{operation_future.future.session.headers['Content-Type']}]")
    operation_result, operation_response = operation_future.result()
except bravado.exception.HTTPBadRequest:
    print("Request failed for {operation_id}! ")
    print(f"URL: {operation_future.future.request.url}")
    raise
except bravado.exception.HTTPNotFound:
    print("Request failed for {operation_id}! ")
    print(f"URL: {operation_future.future.request.url}")
    raise
operation_details = json.loads(operation_result)
# print("\n{} operation_details:\nHTTP method: {}\nAPI url: {}:\n {}\n".format(operation_id, operation_future.operation.http_method, url, pformat(operation_details)))
print(f"Operation {operation_id} finished.")


Operation getInstance starting.
name: Compute-587626604/eric.harris@oracle.com/dbaas/gc3-naac-cdmt-d05-jcsdb/db_1/vm-1/963e740b-052b-469c-849c-37a1d0752b7a
[2018-05-18 15:37:02.013926] DEBUG: gc3_query.lib.bravado.requests_client: input request_params={'method': 'GET', 'url': 'https://compute.uscom-central-1.oraclecloud.com/instance/Compute-587626604%2Feric.harris%40oracle.com%2Fdbaas%2Fgc3-naac-cdmt-d05-jcsdb%2Fdb_1%2Fvm-1%2F963e740b-052b-469c-849c-37a1d0752b7a', 'params': {}, 'headers': {}}
[2018-05-18 15:37:02.015229] DEBUG: gc3_query.lib.bravado.requests_client: url=https://compute.uscom-central-1.oraclecloud.com/instance/Compute-587626604%2Feric.harris%40oracle.com%2Fdbaas%2Fgc3-naac-cdmt-d05-jcsdb%2Fdb_1%2Fvm-1%2F963e740b-052b-469c-849c-37a1d0752b7a parsed_url=ParseResult(scheme='https', netloc='compute.uscom-central-1.oraclecloud.com', path='/instance/Compute-587626604%2Feric.harris%40oracle.com%2Fdbaas%2Fgc3-naac-cdmt-d05-jcsdb%2Fdb_1%2Fvm-1%2F963e740b-052b-469c-849c-37a1d0752



<pre>

      "get" : {
        "tags" : [ "Instances" ],
        "summary" : "Retrieve Details of an Instance",
        "description" : "Retrieves details of the specified instance.<p><b>Required Role: </b>To complete this task, you must have the <code>Compute_Monitor</code> or <code>Compute_Operations</code> role. If this role isn't assigned to you or you're not sure, then ask your system administrator to ensure that the role is assigned to you in Oracle Cloud My Services. See <a target=\"_blank\" href=\"http://www.oracle.com/pls/topic/lookup?ctx=stcomputecs&id=MMOCS-GUID-54C2E747-7D5B-451C-A39C-77936178EBB6\">Modifying User Roles</a> in <em>Managing and Monitoring Oracle Cloud</em>.",
        "operationId" : "getInstance",
        "responses" : {
          "200" : {
            "headers" : {
              "set-cookie" : {
                "type" : "string",
                "description" : "The cookie value is returned if the session is extended"
              }
            },
            "description" : "OK. See <a class=\"xref\" href=\"Status%20Codes.html\">Status Codes</a> for information about other possible HTTP status codes.",
            "schema" : {
              "$ref" : "#/definitions/Instance-response"
            }
          }
        },
        "consumes" : [ "application/oracle-compute-v3+json" ],
        "produces" : [ "application/oracle-compute-v3+json" ],
        "parameters" : [ {
          "name" : "name",
          "in" : "path",
          "description" : "<p>Multipart name of the object.",
          "required" : true,
          "type" : "string"
        }, {
          "name" : "Cookie",
          "in" : "header",
          "type" : "string",
          "description" : "The Cookie: header must be included with every request to the service. It must be set to the value of the set-cookie header in the response received to the POST /authenticate/ call."
        } ]
      }
    }
  },
  
  .
  .
  .
  



    "Instance-response" : {
      "properties" : {
        "account" : {
          "type" : "string",
          "description" : "Shows the default account for your identity domain."
        },
        "attributes" : {
          "additionalProperties" : {
            "type" : "object"
          },
          "type" : "object",
          "description" : "A dictionary of attributes to be made available to the instance. A value with the key \"userdata\" will be made available in an EC2-compatible manner."
        },
        "availability_domain" : {
          "type" : "string",
          "description" : "The availability domain the instance is in"
        },
        "boot_order" : {
          "items" : {
            "type" : "integer"
          },
          "type" : "array",
          "description" : "Boot order list."
        },
        "desired_state" : {
          "type" : "string",
          "description" : "Desired state for the instance. The value can be <code>shutdown</code> or <code>running</code> to shutdown an instance or to restart a previously shutdown instance respectively."
        },
        "disk_attach" : {
          "type" : "string",
          "description" : "A label assigned by the user to identify disks."
        },
        "domain" : {
          "type" : "string",
          "description" : "The default domain to use for the hostname and for DNS lookups."
        },
        "entry" : {
          "type" : "integer",
          "description" : "Optional imagelistentry number (default will be used if not specified)."
        },
        "error_reason" : {
          "type" : "string",
          "description" : "The reason for the instance going to error state, if available."
        },
        "fingerprint" : {
          "type" : "string",
          "description" : "SSH server fingerprint presented by the instance."
        },
        "hostname" : {
          "type" : "string",
          "description" : "The hostname for this instance."
        },
        "hypervisor" : {
          "additionalProperties" : {
            "type" : "object"
          },
          "type" : "object",
          "description" : "A dictionary of hypervisor-specific attributes."
        },
        "image_format" : {
          "type" : "string",
          "description" : "The format of the image."
        },
        "imagelist" : {
          "type" : "string",
          "description" : "Name of imagelist to be launched."
        },
        "ip" : {
          "type" : "string",
          "description" : "IP address of the instance."
        },
        "label" : {
          "type" : "string",
          "description" : "A label assigned by the user, specifically for defining inter-instance relationships."
        },
        "name" : {
          "type" : "string",
          "description" : "Multipart name of the instance."
        },
        "networking" : {
          "additionalProperties" : {
            "type" : "object"
          },
          "type" : "object",
          "description" : "Mapping of <device name> to network specifiers for virtual NICs to be attached to this instance."
        },
        "placement_requirements" : {
          "items" : {
            "type" : "string"
          },
          "type" : "array",
          "description" : "A list of strings specifying arbitrary tags on nodes to be matched on placement."
        },
        "platform" : {
          "type" : "string",
          "description" : "The OS platform for the instance."
        },
        "priority" : {
          "type" : "string",
          "description" : "The priority at which this instance will be run."
        },
        "quota" : {
          "type" : "string",
          "description" : "Not used"
        },
        "relationships" : {
          "items" : {
            "additionalProperties" : {
              "type" : "object"
            },
            "type" : "object"
          },
          "type" : "array",
          "description" : "A list of relationship specifications to be satisfied on this instance's placement"
        },
        "resolvers" : {
          "items" : {
            "type" : "string"
          },
          "type" : "array",
          "description" : "Resolvers to use instead of the default resolvers."
        },
        "reverse_dns" : {
          "type" : "boolean",
          "description" : "Add PTR records for the hostname."
        },
        "shape" : {
          "type" : "string",
          "description" : "A shape is a resource profile that specifies the number of CPU threads and the amount of memory (in MB) to be allocated to an instance."
        },
        "sshkeys" : {
          "items" : {
            "type" : "string"
          },
          "type" : "array",
          "description" : "SSH keys that will be exposed to the instance."
        },
        "start_time" : {
          "type" : "string",
          "description" : "Start time of the instance."
        },
        "state" : {
          "type" : "string",
          "description" : "State of the instance."
        },
        "storage_attachments" : {
          "items" : {
            "type" : "string"
          },
          "type" : "array",
          "description" : "List of dictionaries containing storage attachment Information."
        },
        "tags" : {
          "items" : {
            "type" : "string"
          },
          "type" : "array",
          "description" : "Comma-separated list of strings used to tag the instance."
        },
        "uri" : {
          "type" : "string",
          "description" : "Uniform Resource Identifier"
        },
        "vcable_id" : {
          "type" : "string",
          "description" : "vCable for this instance."
        },
        "vnc" : {
          "type" : "string",
          "description" : "IP address and port of the VNC console for the instance."
        }
      }
    },

</pre>



In [24]:
type(operation_details)
print(operation_details.keys())

dict_keys(['domain', 'placement_requirements', 'ip', 'fingerprint', 'image_metadata_bag', 'site', 'shape', 'imagelist', 'image_format', 'relationships', 'availability_domain', 'networking', 'storage_attachments', 'hostname', 'quota_reservation', 'disk_attach', 'label', 'priority', 'platform', 'state', 'virtio', 'vnc', 'desired_state', 'tags', 'start_time', 'quota', 'entry', 'error_reason', 'sshkeys', 'resolvers', 'account', 'name', 'vcable_id', 'hypervisor', 'uri', 'reverse_dns', 'attributes', 'boot_order'])


#### Insert results in to ~~DB~~  MongoDB

In [25]:
# instances_db.insert(operation_details)

In [26]:
# instances_db.all()[-1]

In [27]:
def global_mongodb_init():
    ###
    db_config = dict(
        host='127.0.0.1',
        port=7117
    )

    # mongoengine.register_connection(alias='core', name='demo_dealership', port=7117)
    mongoengine.register_connection(alias='core', name='instances_mongodb_scratch', **db_config)
global_mongodb_init()

In [28]:
sorted(operation_details)

['account',
 'attributes',
 'availability_domain',
 'boot_order',
 'desired_state',
 'disk_attach',
 'domain',
 'entry',
 'error_reason',
 'fingerprint',
 'hostname',
 'hypervisor',
 'image_format',
 'image_metadata_bag',
 'imagelist',
 'ip',
 'label',
 'name',
 'networking',
 'placement_requirements',
 'platform',
 'priority',
 'quota',
 'quota_reservation',
 'relationships',
 'resolvers',
 'reverse_dns',
 'shape',
 'site',
 'sshkeys',
 'start_time',
 'state',
 'storage_attachments',
 'tags',
 'uri',
 'vcable_id',
 'virtio',
 'vnc']

In [29]:
print(json.dumps(SortedDict(operation_details)))

{"account": "/Compute-587626604/default", "attributes": {"nodelist": "", "cdb": "yes", "service_name": "DBCS_DEFAULT", "flashback_days": "1", "gg": "no", "tfa_autocollect": "no", "managed": "no", "net_security_integrity_checksum_level": "required", "automem": "yes", "fra_mnt": "/u03", "tmpl_sysauxsz": "2000MB", "demo_user": "oracle", "tmpl_logsz": "1000MB", "lsnr_port": "1521", "tde_ks_login": "auto", "pdbss": "no", "sshkeys": ["ssh-dss AAAAB3NzaC1kc3MAAACBAP1/U4EddRIpUt9KnC7s5Of2EbdSPO9EAMMeP4C2USZpRV1AIlH7WT2NWPq/xfW6MPbLm1Vs14E7gB00b/JmYLdrmVClpJ+f6AR7ECLCT7up1/63xhv4O1fnxqimFQ8E+4P208UewwI1VBNaFpEy9nXzrith1yrv8iIDGZ3RSAHHAAAAFQCXYFCPFSMLzLKSuYKi64QL8Fgc9QAAAIEA9+GghdabPd7LvKtcNrhXuXmUr7v6OuqC+VdMCz0HgmdRWVeOutRZT+ZxBxCBgLRJFnEj6EwoFhO3zwkyjMim4TwWeotUfI0o4KOuHiuzpnWRbqN/C/ohNWLx+2J6ASQ7zKTxvqhRkImog9/hWuWfBpKLZl6Ae1UlZAFMO/7PSSoAAACAV0PgmmFiLGK4riYtxxWsAfln+o1AQCI8mlxIQVS0YWSM6wCBIx2byRdNt8n8CtVfktosXpValjTfIDo7X7TUIQ1te5vkFMT75SmBkJCF1BTEWQH88gr3KkBj5i411o/qsOcAZdNpLzeYLxKG4IC3Iz8